In [4]:
# =-=-=-=-=-=
# Read CSV into DataFrame and then create lists
# =-=-=-=-=-=

import pandas
import re

# Create pandas dataframe & lists
colnames = ['author', 'title', 'date' , 'length', 'text']
df = pandas.read_csv('../data/talks_3.csv', names=colnames)
talks = df.text.tolist()


# =-=-=-=-=-=-=-=-=-=-=
# Create citations to identify individual texts
# =-=-=-=-=-=-=-=-=-=-= 

# authors = df.author.tolist()
# dates = df.date.tolist()
# years = [re.sub('[A-Za-z ]', '', item) for item in dates]
# authordate = [author+" "+year for author, year in zip(authors, years)]

In [5]:
# =-=-=-=-=-=
# Clean and Tokenize, then Drop Stopwords
# =-=-=-=-=-=

from nltk.tokenize import WhitespaceTokenizer

# From the Stopwords Notebook:
tokenizer = WhitespaceTokenizer()
stopwords = re.split('\s+', open('../data/tt_stop.txt', 'r').read().lower())

# Loop to tokenize, stop, and stem (if needed) texts.
texts = []
for talk in talks:   
    # clean and tokenize document string
    raw = re.sub(r"[^\w\d'\s]+",'', talk).lower()
    tokens = tokenizer.tokenize(raw)
    # remove stop words from tokens
    stopped_tokens = [i for i in tokens if not i in stopwords]
    # stem tokens
    # stemmed_tokens = [p_stemmer.stem(i) for i in stopped_tokens]
    # add tokens to list
    texts.append(stopped_tokens)

In [6]:
# =-=-=-=-=-=-=-=-=-=-=
# Re-Assemble Texts as Strings from Lists of Words
# =-=-=-=-=-=-=-=-=-=-= 

strungs = []
for text in texts:
    strung = ' '.join(text)
    strungs.append(strung)

In [8]:
# =-=-=-=-=-=
# Get NMF topics
# =-=-=-=-=-=

import sklearn.feature_extraction.text as sk_text
from sklearn.decomposition import NMF

# All our variables are here to make it easier to make adjustments
n_samples = len(strungs)
n_features = 3000
n_topics = 40
n_top_words = 15
# tt_stopwords = open('../data/stopwords_tt.txt', 'r').read().splitlines()

# Get tf-idf features for NMF
vectorizer = sk_text.TfidfVectorizer(max_df = 0.95,
                                        min_df = 0.05,
                                        max_features = n_features)
tfidf = vectorizer.fit_transform(strungs)

# Fit the NMF model
nmf = NMF(n_components = n_topics,
          random_state = 1,
          alpha = 0.1,
          l1_ratio = 0.5).fit(tfidf)
print("Fitting the NMF model with {} topics for {} documents with {} features."
      .format(n_topics, n_samples, n_features))

Fitting the NMF model with 40 topics for 2068 documents with 3000 features.


In [13]:
# =-=-=-=-=-=
# Get NMF printing
# =-=-=-=-=-=

def print_top_words(model, feature_names, n_top_words):
    for topic_id, topic in enumerate(model.components_):
        print('\nTopic {}:'.format(int(topic_id)))
        print(''.join([feature_names[i] + ' ' + str(round(topic[i], 2))
              +', ' for i in topic.argsort()[:-n_top_words - 1:-1]]))

features = vectorizer.get_feature_names()
# print(features)

# This should work now.

print("Topics in NMF model:")
print_top_words(nmf, features, n_top_words) #n_top_words can be changed on the fly

Topics in NMF model:

Topic 0:
just 0.82, really 0.73, actually 0.71, people 0.67, see 0.67, think 0.66, going 0.61, things 0.59, get 0.56, time 0.51, make 0.46, little 0.43, right 0.43, look 0.41, go 0.4, 

Topic 1:
people 0.75, countries 0.53, world 0.53, percent 0.45, country 0.44, global 0.4, government 0.39, money 0.38, dollars 0.37, china 0.34, economic 0.33, economy 0.3, states 0.29, democracy 0.28, social 0.28, 

Topic 2:
women 2.54, men 0.99, girls 0.45, woman 0.41, sex 0.28, female 0.27, gender 0.24, male 0.18, girl 0.15, boys 0.15, violence 0.14, equal 0.11, rights 0.1, young 0.09, husband 0.08, 

Topic 3:
brain 2.68, brains 0.36, arm 0.18, activity 0.18, mental 0.17, human 0.15, body 0.15, memory 0.13, mind 0.12, behavior 0.11, pain 0.11, visual 0.1, animal 0.1, genes 0.1, control 0.09, 

Topic 4:
earth 1.23, planet 1.0, life 0.47, solar 0.47, stars 0.38, sun 0.38, surface 0.36, atmosphere 0.32, star 0.26, system 0.24, moon 0.23, years 0.21, billion 0.16, miles 0.15, size 0

In [16]:
dtm = tfidf.toarray()
doctopic = nmf.fit_transform(dtm) # This is an array
features = vectorizer.get_feature_names()
print(features)

['ability', 'able', 'absolute', 'absolutely', 'accept', 'access', 'accident', 'account', 'achieve', 'act', 'acting', 'action', 'actions', 'active', 'activities', 'activity', 'actual', 'actually', 'add', 'added', 'addition', 'address', 'admit', 'adult', 'adults', 'advanced', 'advantage', 'affect', 'affected', 'afford', 'afraid', 'africa', 'african', 'afternoon', 'age', 'ages', 'ago', 'agree', 'agreed', 'ahead', 'aid', 'aids', 'air', 'al', 'alive', 'allow', 'allowed', 'allowing', 'allows', 'alternative', 'amazing', 'america', 'american', 'americans', 'amount', 'amounts', 'analysis', 'ancient', 'anderson', 'angry', 'animal', 'animals', 'answer', 'answers', 'anti', 'anybody', 'anymore', 'anyway', 'apart', 'appear', 'applause', 'apple', 'applied', 'apply', 'appreciate', 'approach', 'appropriate', 'architecture', 'area', 'areas', 'argue', 'argument', 'arm', 'arms', 'army', 'arrived', 'art', 'article', 'artificial', 'artist', 'artists', 'asia', 'ask', 'asked', 'asking', 'aspect', 'associated'

In [14]:
for topicidx in enumerate(nmf.components_):
    print(topicidx)

(0, array([ 0.09649451,  0.26610491,  0.01969957, ...,  0.06762911,
        0.03002588,  0.05054642]))
(1, array([ 0.01384376,  0.0183957 ,  0.00184777, ...,  0.        ,
        0.0659959 ,  0.04044354]))
(2, array([ 0.        ,  0.        ,  0.        , ...,  0.        ,
        0.09000064,  0.        ]))
(3, array([ 0.0429543 ,  0.03445525,  0.        , ...,  0.        ,
        0.        ,  0.        ]))
(4, array([ 0.        ,  0.02390597,  0.        , ...,  0.        ,
        0.        ,  0.        ]))
(5, array([ 0.        ,  0.        ,  0.        , ...,  0.01543017,
        0.        ,  0.        ]))
(6, array([ 0.       ,  0.       ,  0.       , ...,  0.       ,  0.0261301,  0.       ]))
(7, array([ 0.        ,  0.00600779,  0.        , ...,  0.        ,
        0.        ,  0.        ]))
(8, array([ 0.,  0.,  0., ...,  0.,  0.,  0.]))
(9, array([ 0.,  0.,  0., ...,  0.,  0.,  0.]))
(10, array([ 0.        ,  0.00872869,  0.        , ...,  0.        ,
        0.        ,  0. 

In [17]:
# =-=-=-=-=-=
# Saving output to CSV
# =-=-=-=-=-=

import numpy as np

# Since DOCTOPIC is an array, you can just do:
#      np.savetxt("foo.csv", doctopic, delimiter=",", fmt = "%s")
# http://stackoverflow.com/questions/6081008/dump-a-numpy-array-into-a-csv-file
#
# The above won't give you the names of the files. Instead try this:

topsnum = np.array([list(range(n_topics))])
# topsnum = np.indices((1,n_topics))[1] <-- this is more than we need,
#                                           but it's cool to know more tricks
#
# Two ways to get an array that is of the form [[0,1,2,3,...]].
# It will have the desired dimensions of (1,35) which is what we want

fileheader = np.concatenate((np.array([["citations"]]), topsnum),axis = 1)
authordate = np.array([df.author])

docTopics = np.concatenate((authordate.T, doctopic), axis = 1)
docTopics = np.concatenate((fileheader, docTopics), axis = 0)

np.savetxt("../data/nmf_topics_raw.csv", doctopic, delimiter=",", fmt = "%s")
#np.savetxt("../data/nmf_topics.csv", docTopics, delimiter=",", fmt = "%s")